In [1]:
import numpy as np
import pandas as pd
import pyodbc
import warnings
import re
import nltk
from nltk import ngrams
from collections import defaultdict
import random
from itertools import islice
warnings.filterwarnings('ignore')

from libraries.utils_predict import TextPredictor

In [2]:
conx_string = "DRIVER={ODBC Driver 17 for SQL Server}; SERVER=APCKRMPTMD01PV,41433; Database=Namlos; UID=Namlos_user; PWD=M@!!eL@498#;"
conx = pyodbc.connect(conx_string)
query = '''SELECT [Issue]
  FROM [Namlos].[dbo].[KB_View_Issue_PowerBI]'''


df = pd.read_sql(query, conx)
print(df.shape)
df.head()

(13561, 1)


,Issue
0,PPE diletakkan diluar kotak PPE. Ada barang2 l...
1,tali tas rawan nyangkut/kepleset
2,tray terlalu dekat dengan paha operator
3,Alat elektronik dekat kaki operator
4,Alat elektronik dekat kaki


In [3]:
df.dropna(subset=['Issue'], inplace=True)

In [4]:
def text_clean(text):
    # Remove whitespace
    text = text.strip()
      
    # Remove special characters and punctuation
    text = re.sub(r'[^A-Za-z\s]','', text)
    text = re.sub(r'\b(\w+)\.{3}', r'\1', text)

   
    # Convert to lowercase
    text = text.lower()

    # Remove Number 
    text = re.sub('[0-9]+', '', text)

    # Replace Word 
    text = text.replace('5 S', '5s')
    text = text.replace('5S', '5s')
    text = text.replace('improer','improper')


    # Remove 2 of word repetitif
    pattern = r'\b(\w+)(2)\b'
    text = re.sub(pattern, lambda match: f'{match.group(1)}', text)

    # Separating conjoined words
    word_combine = ['nyangkut','kepleset','tray','kotak','ppe','tali','operator','alat','lantai','part','terjepit','sop','sesuai','kabel',
                    'stagging','meja','menjuntai','penempatan','mesin']
    text = re.sub(r'(' + '|'.join(word_combine) + r')', r'\1 ', text)
   
    return text

df['Issue'] = df['Issue'].apply(text_clean)

In [5]:
predictor = TextPredictor(df['Issue'])
text = "mesin berdebu"
predicted_words = predictor.predict_next_words(text)
print("Predicted next words :")
for word, score in predicted_words:
    print(f"{word}: {score}")
    print(f"{text} {word}")
    print("")

Predicted next words :
dan kotor stop kontak: 0.45477398117054957
mesin berdebu dan kotor stop kontak

kotor stop kontak: 0.31234841109354006
mesin berdebu kotor stop kontak

stop kontak: 0.09686835619023713
mesin berdebu stop kontak

kontak: 0.07894699317400468
mesin berdebu kontak

